In [1]:
import json
import pandas as pd
import requests
import io

C:\Users\hrandriatsp\AppData\Local\Temp\ipykernel_84664\1706038167.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
url = "https://data.ofgl.fr/api/explore/v2.1/catalog/datasets/ofgl-base-communes/exports/csv?select=exer%2C%20montant_en_millions%2C%20reg_name%2C%20dep_name%2C%20epci_name%2C%20epci_code%2C%20qpv%2C%20com_code%2C%20com_name%2C%20type_de_budget%2C%20ptot%2C%20euros_par_habitant%2C%20agregat%2C%20insee&where=%28agregat%20%3D%20%22Encours%20de%20dette%22%20or%20%20agregat%20%3D%20%22Recettes%20de%20fonctionnement%22%20or%20%20agregat%20%3D%20%22Recettes%20d%27investissement%22%20or%20%20agregat%20%3D%20%22D%C3%A9penses%20de%20fonctionnement%22%20or%20%20agregat%20%3D%20%22D%C3%A9penses%20d%27investissement%22%20or%20%20agregat%20%3D%20%22Epargne%20de%20gestion%22%20or%20%20agregat%20%3D%20%22Epargne%20brute%22%20or%20%20agregat%20%3D%20%22Epargne%20nette%22%20or%20agregat%20%3D%20%22Produit%20des%20cessions%20d%27immobilisations%22%20or%20%20agregat%20%3D%20%22Remboursements%20d%27emprunts%20hors%20GAD%22%20or%20%20agregat%20%3D%20%22Annuit%C3%A9%20de%20dette%22%29%20%20and%20ptot%20%3E%3D%2080000%20and%20year%28exer%29%20%3D%202023&limit=-1&timezone=UTC&use_labels=false&epsg=4326"
result = requests.get(url)
df = pd.read_csv(io.StringIO(result.content.decode('utf-8')), sep=";")

In [3]:
df_principal=df.copy()

In [4]:
df_principal['encours_budget_principal'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Encours de dette" else None, axis=1)
df_principal['recette_fonctionnement'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Recettes de fonctionnement" else None, axis=1)
df_principal['recette_investissement'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Recettes d'investissement" else None, axis=1)
df_principal['depense_fonctionnement'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Dépenses de fonctionnement" else None, axis=1)
df_principal['depense_investissement'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Dépenses d'investissement" else None, axis=1)
df_principal['epargne_gestion'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Epargne de gestion" else None, axis=1)
df_principal['epargne_brute'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Epargne brute" else None, axis=1)
df_principal['epargne_nette'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Epargne nette" else None, axis=1)

In [5]:
df_principal['Produit des cessions d’immobilisations'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Produit des cessions d'immobilisations" else None, axis=1)
df_principal['Remboursements d emprunts hors GAD'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Remboursements d'emprunts hors GAD" else None, axis=1)
df_principal['Annuité de la dette'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Annuité de la dette" else None, axis=1)

In [6]:
df_final = df_principal.copy()

In [7]:
df_final = df_final.groupby('insee').agg(
    {
        'exer':'first',
        'reg_name':'first',
        'dep_name':'first',
        'epci_name':'first',
        'epci_code':'first',
        'qpv':'first',
        'com_code':'first',
        'com_name':'first',
        'type_de_budget':'first',
        'ptot':'first',
        'euros_par_habitant':'first',
        'encours_budget_principal': lambda x: x.dropna().sum(),
        'recette_fonctionnement': lambda x: x.dropna().sum(),
        'recette_investissement': lambda x: x.dropna().sum(),
        'depense_fonctionnement': lambda x: x.dropna().sum(),
        'depense_investissement': lambda x: x.dropna().sum(),
        'epargne_gestion': lambda x: x.dropna().sum(),
        'epargne_brute': lambda x: x.dropna().sum(),
        'epargne_nette': lambda x: x.dropna().sum(),
        'Produit des cessions d’immobilisations': lambda x: x.dropna().sum(),
        'Remboursements d emprunts hors GAD': lambda x: x.dropna().sum(),
        'Annuité de la dette': lambda x: x.dropna().sum()
    }
).reset_index()

In [8]:
df_final['capacite_desendetemment'] = df_final.apply(lambda row: row['encours_budget_principal'] / row['epargne_brute'] if row['epargne_brute'] else None, axis=1)
df_final['taux_endetemment'] = df_final.apply(lambda row: row['encours_budget_principal'] / row['recette_fonctionnement'] if row['recette_fonctionnement'] else None, axis=1)
df_final['duree_apparente_dette'] = df_final.apply(lambda row: row['encours_budget_principal'] / row["Remboursements d emprunts hors GAD"] if row["Remboursements d emprunts hors GAD"] else None, axis=1)
df_final['taux_epargne_net'] = df_final.apply(lambda row: row['epargne_nette'] / row['recette_fonctionnement'] if row['recette_fonctionnement'] else None, axis=1)
df_final['taux_epargne_brute'] = df_final.apply(lambda row: row['epargne_brute'] / row['recette_fonctionnement'] if row['recette_fonctionnement'] else None, axis=1)
df_final['taux_epargne_gestion'] = df_final.apply(lambda row: row['epargne_gestion'] / (row['recette_fonctionnement'] - row['Produit des cessions d’immobilisations'] ) if (row['recette_fonctionnement'] - row['Produit des cessions d’immobilisations']) != 0 else None, axis=1)
df_final['ratio_annuité_recette_fonctionnement'] = df_final.apply(lambda row: row['epargne_nette'] / row['Annuité de la dette'] if row['Annuité de la dette'] else None, axis=1)

In [9]:
df_final['epargne_gestion_habitant'] = df_final.apply(lambda row: row['epargne_gestion']*1000000  / row['ptot'] if row['ptot'] else None, axis=1)
df_final['epargne_brute_habitant'] = df_final.apply(lambda row: row['epargne_brute']*1000000  / row['ptot'] if row['ptot'] else None, axis=1)
df_final['epargne_nette_habitant'] = df_final.apply(lambda row: row['epargne_nette']*1000000 / row['ptot'] if row['ptot'] else None, axis=1)
df_final['budget_habitant'] = df_final.apply(lambda row: row['encours_budget_principal']*1000000  / row['ptot'] if row['ptot'] else None, axis=1)

In [10]:
df_final.to_csv('donnes_financieres_communnes.csv')

In [11]:
import json
import pandas as pd

def get_json_finance_communes(dataframe,epci_code, exer):
  """Retrieves data from the DataFrame and returns it as a JSON.

  Args:
    epci_code: The EPCI code to filter by.
    exer: The exer to filter by.

  Returns:
    A JSON object containing the data, or an error message if data is not found or if inputs are invalid.
  """
  try:
    exer = int(exer)  # Convert exer to integer
  except ValueError:
    return json.dumps({"error": "Invalid exer value. Please provide a valid year."})

  filtered_df = df_final[(dataframe['insee'] == epci_code) & (dataframe['exer'] == exer)]

  if filtered_df.empty:
    return json.dumps({"error": f"No data found for insee: {epci_code} and exer: {exer}"})

  data_dict = filtered_df.iloc[0].to_dict()

  # Handle potential NaN values
  for key, value in data_dict.items():
      if pd.isnull(value):
          data_dict[key] = None  # or any other desired representation for NaN

  return json.dumps(data_dict, ensure_ascii=False).encode('utf8')

test = get_json_finance_communes(df_final,21231, 2023)
print(test.decode())  # This would print the JSON string (if data is found).

{"insee": 21231, "exer": 2023, "reg_name": "Bourgogne-Franche-Comté", "dep_name": "Côte-d'Or", "epci_name": "Dijon Métropole", "epci_code": 242100410, "qpv": "Oui", "com_code": 21231, "com_name": "Dijon", "type_de_budget": "Budget principal", "ptot": 162454, "euros_par_habitant": 1074.256759082571, "encours_budget_principal": 110.28409409, "recette_fonctionnement": 206.74528794, "recette_investissement": 12.51211978, "depense_fonctionnement": 174.51730754, "depense_investissement": 62.80699139, "epargne_gestion": 35.93986893, "epargne_brute": 32.2279804, "epargne_nette": 21.3208081, "Produit des cessions d’immobilisations": 1.30986126, "Remboursements d emprunts hors GAD": 10.9071723, "Annuité de la dette": 0, "capacite_desendetemment": 3.4219982984102844, "taux_endetemment": 0.5334297830381788, "duree_apparente_dette": 10.111153565438771, "taux_epargne_net": 0.10312596873398903, "taux_epargne_brute": 0.1558825389498258, "taux_epargne_gestion": 0.17494484525291906, "ratio_annuité_recet

In [12]:
import json
import os

def generate_json_per_row(dataframe):
  """Génère un fichier JSON pour chaque ligne du dataframe.

  Args:
    dataframe: Le dataframe d'entrée pandas.
  """

  # Crée un répertoire pour stocker les fichiers JSON s'il n'existe pas
  os.makedirs('json_files', exist_ok=True)

  for index, row in dataframe.iterrows():
    # Convertit la ligne en un dictionnaire
    row_dict = row.to_dict()

    # Gère les valeurs NaN potentielles
    for key, value in row_dict.items():
        if pd.isnull(value):
            row_dict[key] = None  # ou toute autre représentation souhaitée pour NaN

    # Crée le dictionnaire au format souhaité
    json_data = {}
    for column, value in row_dict.items():
        json_data[column] = value

    json_data_final = {}
    json_data_final['indicateurs_financiers'] = json_data

    # Génère le nom du fichier en utilisant le nom de la ville formaté
    filename = f"json_files/{row_dict['com_name']}.json"  # Utilise le nom formaté comme nom de fichier

    # Écrit le dictionnaire dans un fichier JSON
    with open(filename, 'w', encoding='utf-8') as f:
      json.dump(json_data_final, f, ensure_ascii=False, indent=4)

  print(f"Fichiers JSON générés dans le répertoire 'json_files'.")

# Appelle la fonction pour générer les fichiers JSON
generate_json_per_row(df_final)

ModuleNotFoundError: No module named 'unidecode'